In [14]:
#!/usr/bin/env python3

# Jorge Ramirez
# 2024 09 02
# Subdividir el excel de compras en 4 para informar a las sucursales.



import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns    
import os
import openpyxl
import shutil

from openpyxl import load_workbook
import sys

os.chdir("c:/Users/jorge_ramirez/OneDrive - Previsora del Parana SRL/Escritorio/0_proyectos Python/Altas_Bajas_cambio/")
diractual = os.getcwd() # para conocer el Current Working Directory
print(diractual)


# ________________________________   ______________________________
# Importar un excel desde una ruta especifica

from openpyxl import load_workbook
from openpyxl.utils import get_column_letter
from pprint import pprint


#df_quiebres = pd.read_excel(r"C:\Users\jorge_ramirez\OneDrive - Previsora del Parana SRL\Escritorio\0_proyectos Python\prediccion_Ventas\02_VER_QUIEBRES.xlsx", sheet_name="Sheet1") # Nombres de columnas 


# Función para ver los ultimos 3 archivos de una carpeta
from os import scandir, getcwd

def get_newest_files(folder_path, num_files=3):
    file_timestamps = [(file.name, file.stat().st_mtime) for file in scandir(folder_path) if file.is_file()]
    sorted_files = sorted(file_timestamps, key=lambda x: x[1], reverse=True)
    newest_files = sorted_files[:num_files]
    return newest_files

# Especifica la ruta de la carpeta 
folder_path = "files_import/"
nf = get_newest_files(folder_path, num_files=3)

print(nf)


nombre_excel_cambios = input("Ingresa el nombre del Excel cambios. nf00 - {}:".format(nf[0][0]))

while os.path.exists("files_import/{}.xlsx".format(nombre_excel_cambios)) == False:
   nombre_excel_ofertas = input("Nombre error, Ingresé nombre del Excel: ")
    
nombre_excel = "files_import/{}.xlsx".format(nombre_excel_cambios)

c:\Users\jorge_ramirez\OneDrive - Previsora del Parana SRL\Escritorio\0_proyectos Python\Altas_Bajas_cambio
[('Estado Productos 07-10-24.xlsx', 1728312448.9593577), ('Estado Productos 01-10-24.xlsx', 1728070940.055994), ('Estado Productos 09-09-24.xlsx', 1725896100.0)]


In [15]:

# Ruta del archivo Excel
file_path = nombre_excel

# Leer las hojas en dataframes
df_super = pd.read_excel(file_path, sheet_name='SUPER', header=4)
df_electro = pd.read_excel(file_path, sheet_name='ELECTRO', header=4)


In [16]:
def Cambia_nombre_STATUS(data_frame):
    columns_names = data_frame.columns[3:]
    cambia_nombre = input("desea cambiar el nombre a STATUS, de {}".format(columns_names))
    data_frame = data_frame.rename(columns={"{}".format(cambia_nombre):"STATUS"})
    return data_frame

df_electro = Cambia_nombre_STATUS(df_electro)

df_super = Cambia_nombre_STATUS(df_super)

In [17]:
# reemplazo el texto en la columna status, para emprolijar

df_electro['STATUS'] = df_electro['STATUS'].replace('No existe - Producto nuevo', 'Producto nuevo')
df_super['STATUS'] = df_super['STATUS'].replace('No existe - Producto nuevo', 'Producto nuevo')

In [18]:
# Quitar espacios en blanco mal puestos y poner mayuscula en la primer letra

def capitalice_text (columna):
    '''Quitar espacios en blanco mal puestos y poner mayuscula en la primer letra'''
    columna_nueva = []
    for texto in columna:
        texto1 = texto.strip().lower().capitalize()
        columna_nueva.append(texto1)
    return columna_nueva


df_super['STATUS'] = capitalice_text(df_super['STATUS'])
df_electro['STATUS'] = capitalice_text(df_electro['STATUS'])

In [19]:
df_super['STATUS'].unique()

array(['Cambió', 'Producto nuevo'], dtype=object)

In [20]:


# Subdividir los dataframes según la variable 'DISCONTINUIDAD'
super_art_nuevos = df_super[df_super['STATUS'] == 'Producto nuevo']
super_art_cambios = df_super[df_super['STATUS'] != 'Producto nuevo']

electro_art_nuevos = df_electro[df_electro['STATUS'] == 'Producto nuevo']
electro_art_cambios = df_electro[df_electro['STATUS'] != 'Producto nuevo']

# Ver el tamaño de una lista de dataframes, con los nombres como string
def lists_size(list_names_dfs):
    '''Ver el tamaño de una lista de nombres en formato str de dataframes''' 
    for i in list_names_dfs:
        df = globals()[i]  # Obtener el dataframe por su nombre
        tamaño = len(df)  # Obtener el tamaño del dataframe
        print(f"Nombre: {i}, Tamaño: {tamaño}")

lists_size(['super_art_nuevos', 'super_art_cambios', 'electro_art_nuevos', 'electro_art_cambios'])

# Exportar los dataframes a archivos Excel
super_art_nuevos.to_excel('SUPER_Art_Nuevos.xlsx', index=False)
super_art_cambios.to_excel('SUPER_Art_Cambios.xlsx', index=False)
electro_art_nuevos.to_excel('Electro_Art_Nuevos.xlsx', index=False)
electro_art_cambios.to_excel('Electro_Art_Cambios.xlsx', index=False)

# Crear gráficos
def create_bar_chart(df, title, filename):
    if not df.empty:
        df['MARCA'].value_counts().plot(kind='bar')
        plt.title(title)
        plt.xlabel('Marca')
        plt.ylabel('Cantidad')
        plt.savefig(filename, dpi=100, bbox_inches='tight')
        plt.close()
    else:
        print(f"No hay datos para {title}, no se creó el gráfico.")



Nombre: super_art_nuevos, Tamaño: 5
Nombre: super_art_cambios, Tamaño: 17
Nombre: electro_art_nuevos, Tamaño: 13
Nombre: electro_art_cambios, Tamaño: 1


In [21]:
# Crear gráficos para cada dataframe
create_bar_chart(super_art_nuevos, 'SUPER - Productos Nuevos', '01_SUPER_Art_Nuevos.png')
create_bar_chart(super_art_cambios, 'SUPER - Productos Cambiados', '02_SUPER_Art_Cambios.png')
create_bar_chart(electro_art_nuevos, 'ELECTRO - Productos Nuevos', '01_Electro_Art_Nuevos.png')
create_bar_chart(electro_art_cambios, 'ELECTRO - Productos Cambiados', '02_Electro_Art_Cambios.png')

print("Los archivos Excel y gráficos se han exportado correctamente.")


Los archivos Excel y gráficos se han exportado correctamente.
